In [1]:
import pandas as pd
import ast

In [2]:
df_stops_1980 = pd.read_csv('../1980-network/stops_df_1980-final.csv')
df_stops_1982 = pd.read_csv('../1982-network/stops_df_1982-final.csv')
df_stops_1984 = pd.read_csv('../1984-network/stops_df_1984-final.csv')

combined_df = pd.concat([df_stops_1980, df_stops_1982, df_stops_1984], ignore_index=True)

df_1989 = pd.read_csv('stops_df_1989-initial.csv')

In [3]:
df_1989.head(3)

,Unnamed: 0,stop_name,line_count,type,in_lines
0,0,Mahlsdorfer Allee,1,bus,{'8'}
1,1,Summter Strasse,1,bus,{'8'}
2,2,Hultschiner Damm,1,bus,{'8'}


In [4]:
df_1989.drop(columns=['Unnamed: 0'], inplace=True)
df_1989["location"] = ""
df_1989["identifier"] = ""
df_1989["stop_id_other"] = ""
df_1989["previous_in_lines"] = ""
df_1989["stop_description"] = ""

In [5]:
combined_df.head(3)

,Unnamed: 0,stop_id,stop_name,type,location,in_lines,identifier
0,0,19800,Achtermannstrasse,bus,"52.57652468032656, 13.418517187666131",['52'],NaN
1,1,19801,Adalbertstrasse,bus,"52.50548699996121, 13.42221404498021","['78', '75', '78E']",NaN
2,2,19802,Adamstrasse,bus,"52.521029117218866, 13.19780699419913","['97', '34']",NaN


I have two dataframes df_long and df_short, I want to add the information from df_long columns combined_location and combined_identifier to df_short if there is a match between the original_name column from df_long with the stop_name column of df_short. I have to create combined_location and combined_identifier columns in df_short first

In [6]:
for index, row in df_1989.iterrows():
 
    # and row["in_lines"] is in df_stops_total["in_lines
    condition_in_lines = any(all(value in ast.literal_eval(line) for value in ast.literal_eval(row["in_lines"])) for line in combined_df["in_lines"].values)


    if row["stop_name"] in combined_df["stop_name"].values and condition_in_lines:
        # Find matching rows in df_1960
        matching_rows = combined_df[combined_df["stop_name"] == row["stop_name"]]
        
        # Check if those two rows have matching row["type"] values
        matching_rows = matching_rows[matching_rows["type"] == row["type"]]
        
        # Check if there are any matching rows
        if not matching_rows.empty:
            df_1989.at[index, 'location'] = '+ '.join(matching_rows['location'].astype(str).values)
            df_1989.at[index, 'identifier'] = '+ '.join(matching_rows['identifier'].astype(str).values)
            df_1989.at[index, 'stop_id_other'] = '+ '.join(matching_rows['stop_id'].astype(str).values)
            df_1989.at[index, 'previous_in_lines'] = '+ '.join(matching_rows['in_lines'].astype(str).values)
        else:
            # Handle the case where there are no matching rows in df_1960
            df_1989.at[index, 'location'] = ''
            df_1989.at[index, 'identifier'] = ''
            df_1989.at[index, 'stop_description'] = ''
            df_1989.at[index, 'stop_id_other'] = ''
            df_1989.at[index, 'previous_in_lines'] = ''


In [7]:
# Function to split string values containing '+' and convert to a unique string
def split_values(cell_value):
    unique_values = sorted(set(element.strip() for element in cell_value.split('+')))
    return '+'.join(unique_values)

# Apply the function to each cell in the specified columns
df_1989['location'] = df_1989['location'].apply(split_values)
df_1989['identifier'] = df_1989['identifier'].apply(split_values)

In [8]:
df_1989.drop(columns=['line_count', "stop_id_other"], inplace=True)

In [9]:
# Function to concatenate unique values in other columns
def concat_unique_values(series):
    unique_values = set(series)
    return '+ '.join(unique_values)

# Group by specified columns and aggregate the other columns
df_1989 = df_1989.groupby(['stop_name', 'type', 'in_lines', 'location']).agg({
    'identifier': concat_unique_values,
    'previous_in_lines': concat_unique_values,
    'stop_description': concat_unique_values
}).reset_index()

In [10]:
df_1989

,stop_name,type,in_lines,location,identifier,previous_in_lines,stop_description
0,Achtermannstrasse,bus,{'52'},"52.57652468032656, 13.418517187666131",nan,['52']+ {'52'}+ {'52'},
1,Adalbertstrasse,bus,{'78'},"52.50548699996121, 13.42221404498021",nan,"['78', '75', '78E']+ {'78', '78E'}+ {'75', '78...",
2,"Ahrensfelde, Lindenberger Strasse",bus,{'41'},"52.57534681022296, 13.576454584108735",nan,{'41'},
3,Alexanderplatz,bus,{'78'},"52.5219449,13.4143847",nan,"['78', '9', '78E', '112']+ {'78', '9', '78E'}+...",
4,Allee der Kosmonauten,bus,"{'48', '53'}","52.54468067619617, 13.56457587733865",nan,"['53']+ {'53'}+ {'53', '48'}",
...,...,...,...,...,...,...,...
267,Zentralschule,bus,{'61'},"52.43465069286089, 13.703692212644038",nan,['61']+ {'61'}+ {'61'},
268,"Zepernick, Kirche",bus,{'51'},"52.65490229314837, 13.542260763377884",nan,['51']+ {'51'}+ {'51'},
269,Zepernicker Strasse,bus,{'51'},"52.6443665848066, 13.523488908135207",nan,{'51'}+ {'51'},
270,Zimmermannstrasse,bus,{'35'},"52.486610959320956, 13.5603667983302",nan,['35']+ {'35'}+ {'35'},


In [11]:
df_1989.to_csv('1989-stops-combined.csv')